In this notebook we tried to build an LDA model on dataset. We assume the dataset is preprocessed: It was removed stop words and contains only meaningful contents. There are only two columns of the input dataset:

In [2]:
import os
import pandas as pd 
import numpy as np 
import random
import re
import matplotlib as plt
import seaborn as sns
import gensim # this is LDA package

In [4]:
# This block read dataset and look at its sample

df = pd.read_csv('nyt.csv')
df.sample(10)

,Unnamed: 0.1,Unnamed: 0,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,...,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name,date
3969,4143,57304,after firing the shots the suspect said “free ...,https://www.nytimes.com/2023/12/07/nyregion/te...,"After firing the shots, the suspect said, “Fre...",a man fired a shotgun into the air outside an ...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Metro,New York,"{'original': 'By Grace Ashford and Jay Root', ...",News,nyt://article/250b53b5-1dec-5b00-9629-4cd8dc6a...,764.0,nyt://article/250b53b5-1dec-5b00-9629-4cd8dc6a...,NaN,2023-12-07 22:58:03+00:00
494,526,53687,the elusive vocalist and producer performed hi...,https://www.nytimes.com/2023/05/04/arts/music/...,The elusive vocalist and producer performed hi...,when a collection of jai paul demos leaked onl...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Culture,Arts,"{'original': '', 'person': [], 'organization':...",News,nyt://article/816d9738-3d2c-5e38-a86b-aa71b1e0...,239.0,nyt://article/816d9738-3d2c-5e38-a86b-aa71b1e0...,Music,2023-05-04 15:24:15+00:00
1948,2051,55212,in two memoirs magazine articles and a times e...,https://www.nytimes.com/2023/05/16/books/amy-s...,"In two memoirs, magazine articles and a Times ...",amy silverstein a celebrated writer whose two ...,A,21.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Obits,Books,"{'original': 'By Alex Williams', 'person': [{'...",Obituary (Obit),nyt://article/89ed7eec-657b-57d7-819f-6e21c1f2...,1176.0,nyt://article/89ed7eec-657b-57d7-819f-6e21c1f2...,NaN,2023-05-16 18:56:39+00:00
2882,3017,56178,the police department will send them up in bro...,https://www.nytimes.com/2023/09/01/nyregion/dr...,The Police Department will send them up in Bro...,the beating of drums the bleating of horns and...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Metro,New York,"{'original': 'By Claire Fahy', 'person': [{'fi...",News,nyt://article/26d18062-d394-55e2-baea-4632f281...,609.0,nyt://article/26d18062-d394-55e2-baea-4632f281...,NaN,2023-09-01 15:14:54+00:00
2373,2487,55648,addressing an arab league summit in saudi arab...,https://www.nytimes.com/2023/05/19/world/europ...,Addressing an Arab League summit in Saudi Arab...,president volodymyr zelensky of ukraine implor...,A,5.0,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Foreign,World,"{'original': 'By Vivian Nereim', 'person': [{'...",News,nyt://article/7cbd7c63-c42d-5db9-9622-c10432e4...,1263.0,nyt://article/7cbd7c63-c42d-5db9-9622-c10432e4...,Europe,2023-05-19 10:04:31+00:00
5401,5648,58809,“maestro” isn t the first time a supersize sni...,https://www.nytimes.com/2023/12/20/movies/maes...,“Maestro” isn’t the first time a supersize sni...,in august the first trailer for “maestro” a bi...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Culture,Movies,"{'original': 'By Sarah Bahr', 'person': [{'fir...",News,nyt://article/4d1e6b39-9615-57af-974a-6f03c2ff...,1020.0,nyt://article/4d1e6b39-9615-57af-974a-6f03c2ff...,NaN,2023-12-20 10:01:41+00:00
4315,4510,57671,a reader seeks guidance for navigating the ove...,https://www.nytimes.com/2023/12/11/style/susta...,A reader seeks guidance for navigating the ove...,the irony of finding the perfect pair of jeans...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",...,article,Styles,Style,"{'original': 'By Vanessa Friedman', 'person': ...",News,nyt://article/e8d53914-fb4e-5f7b-82e9-b2f4b911...,664.0,nyt://article/e8d53914-fb4e-5f7b-82e9-b2f4b911...,NaN,2023-12-11 10:02:38+00:00
4224,4415,57576,her solution a packaging business that sells i...,https://www.nytimes.com/2023/12/10/books/dhoni...,Her solution? A packaging business that sells ...,one ev

In [8]:
# For the test purpose, we use only abstract here.

X = df.abstract

X.sample(10)

2092    president biden selected shalanda young and st...
2982    lisa donovan weaves her own story into this re...
5086    the tree which has been decorated each year be...
5655    tammy murphy is backed by some new jersey demo...
5620      plus how wealthy was the family in “home alone”
4597    we have the tools we need to create a world wh...
2355    seven brand new and time tested books about sl...
5603    a huge ice sheet appears to have melted about ...
889     everyone gets mad about spoilers but science s...
433     a decorated writer in the s nancy hale analyze...
Name: abstract, dtype: object

## Tokenization and Further Data preparing

The book articles are removed from the initial list of book properties
Punctuation is removed
Article strings are turned into tokens, and stop words are removed
Numeric tokens were removed
Bigrams are created
Words are lemmatized to reduce words of similar roots to similar tokens
English first names are removed, as these are often very prevalent in articles (note that bigrams of full names are retained)
Token amounts per article are rounded off at an exponentially increasing rate (we'll more explain when we get there)
Tokens of that are less than four characters are removed
Only tokens that appear in at least ten articles are kept
Those tokens that appear in more than 15% of all articles will be removed

In [175]:
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

In [16]:
X_train, X_val = train_test_split(X, test_size=0.2, shuffle= True, random_state= 216)

In [196]:
# We start by tokenize the texts

print("Extracting features from the training data using a Count vectorizer")

vectorizer_count = CountVectorizer(#sublinear_tf=True, 
                                   max_df=0.15, #remove words that have a document frequency strictly higher than 15%
                                   min_df=0.001, #remove words that have a document frequency strictly lower than 0.1%
                                   stop_words='english', #removing stop words
                                   ngram_range=(1,2), #unigrams and bigrams are generated
                                   max_features=2**14)

#We train the vectorizer using X_train
X_train_count = vectorizer_count.fit_transform(X_train)
X_val_count = vectorizer_count.transform(X_val) # Extracting features from the test data using the same vectorizer
#X_test_tfidf = vectorizer_tfidf.fit_transform(X_test)



Extracting features from the training data using a Count vectorizer


In [191]:
# We start by tokenize the texts

print("Extracting features from the training data using a TF-IDF vectorizer")

vectorizer_tfidf = TfidfVectorizer(sublinear_tf=True, 
                                   max_df=0.15, #remove words that have a document frequency strictly higher than 15%
                                   min_df=0.001, #remove words that have a document frequency strictly lower than 0.1%
                                   stop_words='english', #removing stop words
                                   ngram_range=(1,2), #unigrams and bigrams are generated
                                   max_features=2**14)

#We train the vectorizer using X_train
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_val_tfidf = vectorizer_tfidf.transform(X_val) # Extracting features from the test data using the same vectorizer
#X_test_tfidf = vectorizer_tfidf.fit_transform(X_test)

# To visualize bow we transform them into a dataframe
X_train_tok = pd.DataFrame(X_train_tfidf.toarray(), index = X_train, columns= vectorizer_tfidf.get_feature_names_out())
X_val_tok = pd.DataFrame(X_val_tfidf.toarray(), index = X_val, columns= vectorizer_tfidf.get_feature_names_out())

X_train_tok_empty = X_train_tok.loc[(X_train_tok== 0).all(axis= 1)]
X_val_tok_empty = X_val_tok.loc[(X_val_tok== 0).all(axis= 1)]

print(len(X_train_tok_empty)/len(X_train_tok))

print(len(X_val_tok_empty)/len(X_val_tok))

# There are some empty tokenizations, we should definitely look into them

Extracting features from the training data using a TF-IDF vectorizer
0.004796623177283192
0.004604758250191865


In [78]:
X_val_tok_empty
# So it seems like we have some non-English articles (ruled out by min_df) and also some short and ambiguous abstratc. 
# For now we will rule them out

,ability,able,abortion,abroad,abuse,access,according,according new,account,accounts,...,york,york city,york times,yorkers,young,young people,younger,youth,zelensky,zelensky ukraine
abstract,,,,,,,,,,,,,,,,,,,,,
we have more incentives to be heterodox,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
also soothing and satisfying crisp gnocchi with sausage and peas soy sauce noodles with cabbage and fried eggs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
asher seems to be losing his grip except when what in his grip is a bunch of nails,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
solving brad wiegmann crossword requires prudence,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
and supersize your almond croissant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
four insiders on where to go for rooftop drinks treasure hunting and more,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
# Look at unique tokens and articles
print(len(vectorizer_tfidf.get_feature_names_out()))

print(len(vectorizer_count.get_feature_names_out()))

#they are the same

2591
2591


In [193]:
# Look at maximal numbers of tokens
np.max(np.count_nonzero(X_train_count.toarray(),axis=1))
#np.min(np.count_nonzero(X_val_tfidf,axis=1))


32

## Model prep
In this part we create our dictionary 

In [ ]:
from gensim import corpora, models, similarities

In [188]:
# Make a dictionary for our bow

dictionary_tfidf = vectorizer_tfidf.get_feature_names_out()
dictionary_count = vectorizer_count.get_feature_names_out()

len(dictionary_count[np.invert(np.in1d(dictionary_count,dictionary_tfidf))])

# this line shows that the two dictionary are exactly the same!

0

In [159]:
def tok_report(bow, num):
    article = bow[num] 
    arg = article.nonzero()
    print(dictionary[arg])

tok_report(X_val_tfidf, 0)

['films' 'original' 'played' 'sports' 'tale' 'wasn' 'winning']


In [173]:

from gensim.models import LdaModel

In [198]:
type(X_train_count)

scipy.sparse._csr.csr_matrix

In [206]:
lda = models.LdaModel(X_train_count, #the bow of corpus
                      num_topics=4) # We should do cv to optimize this value 
                                  

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().